# U-Net trained on the MoNuSeg data set

In [1]:
import random
import json
from itertools import cycle
import xml.etree.ElementTree as ET
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from PIL import Image
from PIL import Image, ImageDraw
from skimage.draw import polygon
from skimage.io import imread, imshow
from skimage.transform import resize
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
from keras.utils import Sequence
from keras.models import Model, load_model
from keras.layers import Input, multiply
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import pandas as pd
import tensorflow as tf
#import wandb
#from wandb.keras import WandbCallback

import utils
import monuseg_utils

from augmented_sequence import AugmentedSequence

In [2]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 3
BATCH_SIZE = 5

## Setting up training, validation and test groups
Should use all MoNuSeg data, not only the test data

In [3]:
monuseg_patinfo = pd.read_csv(Path(os.getcwd()).parent / 'data/monuseg/patient_information.csv')

monuseg_train_ids = monuseg_patinfo[monuseg_patinfo['training'] == 'yes']['patient_id']
monuseg_test_ids = monuseg_patinfo[monuseg_patinfo['training'] == 'no']['patient_id']
monuseg_train_ids, monuseg_val_ids = utils.val_split(monuseg_train_ids)

## Data augmentation

In [ ]:
seq = iaa.Sequential([
    iaa.CropToFixedSize(width=round(IMG_WIDTH*1.5), height=round(IMG_HEIGHT*1.5)),
    iaa.flip.Flipud(p=0.5),
    iaa.flip.Fliplr(p=0.5),
    iaa.Affine(rotate=(-45,45)),
    iaa.CropToFixedSize(width=IMG_WIDTH, height=IMG_HEIGHT, position="center"),
    iaa.Sometimes(0.2, iaa.SigmoidContrast(gain=(3, 10), cutoff=(0.4, 0.6))),
    iaa.Sometimes(0.2, iaa.GaussianBlur(sigma=(0.0, 3.0)))
    #iaa.Sometimes(0.2, iaa.EdgeDetect((0,0.5)))
])

val_sequential = iaa.Sequential([
    iaa.CropToFixedSize(width=round(IMG_WIDTH*1.5), height=round(IMG_HEIGHT*1.5)),
    iaa.flip.Flipud(p=0.5),
    iaa.flip.Fliplr(p=0.5),
    iaa.Affine(rotate=(-45,45)),
    iaa.CropToFixedSize(width=IMG_WIDTH, height=IMG_HEIGHT, position="center"),
])

train_seq = AugmentedSequence(monuseg_train_ids, BATCH_SIZE, seq, IMG_WIDTH, IMG_HEIGHT)
val_seq = AugmentedSequence(monuseg_val_ids, BATCH_SIZE, val_sequential, IMG_WIDTH, IMG_HEIGHT)

## Train U-Net
The U-Net, along with the weight map loss function, was implemented by [jaidevd](https://jaidevd.github.io/posts/weighted-loss-functions-for-instance-segmentation/).

In [ ]:

tf.compat.v1.disable_eager_execution()
from weighted_loss_unet import make_weighted_loss_unet, my_loss

#wandb.init(project="swebcg91rt-net")


model = make_weighted_loss_unet((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), 1)
model.compile(optimizer='adam', loss=my_loss, metrics=[tf.keras.metrics.MeanIoU(num_classes=2)])
model.fit(train_seq, validation_data=val_seq, epochs=150)#, callbacks=[WandbCallback()]) 


# Save model to wandb
#model.save(os.path.join(wandb.run.dir, "model.h5"))

In [ ]:
model.save('./models/unet_model.h5')


## Predict on test set

In [ ]:
test_seq = iaa.Sequential([iaa.CropToFixedSize(width=IMG_WIDTH, height=IMG_HEIGHT)])
test_gen = AugmentedSequence(monuseg_test_ids, BATCH_SIZE, test_seq, IMG_WIDTH, IMG_HEIGHT)

In [ ]:

import motplotlib.gridspec as gridspec
for i in range(len(test_gen)):
    xb, yb